In [1]:
import os
from openai import OpenAI
import httpx

import inspect
from diskcache import Cache
from pydantic import BaseModel

from typing import Any
import jupyter_black
import requests
from markdownify import markdownify as md

jupyter_black.load()

DEEPSEEK_API_KEY = os.environ["DEEPSEEK_API_KEY"]
DEEPSEEK_BASE_URL = "https://api.deepseek.com"
MODEL = "deepseek-chat"
cache = Cache("./cache")
client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url=DEEPSEEK_BASE_URL)

In [2]:
def to_openai_tool(func: callable) -> dict[str, Any]:
    sig = inspect.signature(func)
    if sig.parameters:
        param = next(iter(sig.parameters.values()))
        param_type = param.annotation
        schema = param_type.model_json_schema()
    else:
        param_type = None
        schema = {"type": "object", "properties": {}, "required": []}

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (param_type.__doc__ if param_type else func.__doc__ or ""),
            "parameters": schema,
        },
    }


@cache.memoize(expire=3600)
def llm_call(
    messages: list[dict[str, str]], tools: list[dict[str, Any]], model: str
) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools if tools else None,
        temperature=0.0,
        tool_choice="auto",
    )

    message = response.choices[0].message

    if hasattr(message, "reasoning_content"):
        del message.reasoning_content

    return message

In [3]:
class SearchWeb(BaseModel):
    """Search the web for a given query."""

    query: str
    max_results: int = 10


def search_web(data: SearchWeb) -> str:
    base_url = "https://api.marginalia.nu/{key}/search/{query}"
    url = base_url.format(key="public", query=data.query)
    url += f"?count={data.max_results}"

    rsp = httpx.get(url)
    rsp.raise_for_status()

    results = rsp.json()["results"]
    return str(results)


class ReadWebsite(BaseModel):
    """Read the content of a website and return it as text. (useful for further research)"""

    url: str


def read_website(data: ReadWebsite) -> str:

    html_content = requests.get(data.url).text
    return md(html_content)


TOOLS = [
    to_openai_tool(search_web),
    to_openai_tool(read_website),
]
FUNC_TYPES = {
    search_web: SearchWeb,
    read_website: ReadWebsite,
}

In [4]:
class ReACTAgent:
    def __init__(
        self,
        model: str,
        system: str,
        tools: list[dict[str, Any]] | None = None,
        max_iters: int = 10,
    ):
        self.model = model
        self.tools = tools or []
        self.messages: list[dict[str, str]] = []
        if system:
            self.messages.append({"role": "system", "content": system})

        self.max_iters = max_iters

    def __call__(self, content: str, verbose: bool) -> str:
        self.messages.append({"role": "user", "content": content})

        iterations = 0
        while True:
            iterations += 1
            message = self._send()
            self.messages.append(message)

            tool_calls = getattr(message, "tool_calls", None)
            if not tool_calls:
                return message.content

            for call in tool_calls:
                func = globals()[call.function.name]
                args_model = FUNC_TYPES[func]
                args = (
                    args_model.model_validate_json(call.function.arguments)
                    if call.function.arguments != "{}"
                    else None
                )
                print(f"** CALLING FUNCTION {func.__name__} **")
                print(f"*** ARGS ***\n{args}\n")

                result = func(args) if args else func()

                if verbose is True:
                    print(f"*** RESULT ***\n{result}\n***")

                self.messages.append(
                    {
                        "role": "tool",
                        "tool_call_id": call.id,
                        "content": result,
                    }
                )

            if iterations >= self.max_iters:
                raise Exception("Max iterations reached")

    def _send(self):
        return llm_call(self.messages, self.tools, self.model)

In [5]:
system_prompt = """
You are a helpful assistant who can answer multistep questions by sequentially calling functions. 

Follow a pattern of:
- THOUGHT (reason step-by-step about which function to call next)
- ACTION (call a function as a next step towards the final answer) 
- OBSERVATION (output of the function)

Reason step by step which actions to take to get to the answer. 

Only call functions with arguments coming verbatim from the user or the output of other functions.
"""

question = "What is the population of the city of Ancona as of 2025?"

In [6]:
bot = ReACTAgent(system=system_prompt, tools=TOOLS, model="deepseek-chat")
response = bot(question, verbose=False)
print("Response:", response)

** CALLING FUNCTION search_web **
*** ARGS ***
query='population of Ancona Italy 2025' max_results=10

** CALLING FUNCTION read_website **
*** ARGS ***
url='https://population.city/italy/ancona/'

Response: The last known population of Ancona, Italy, was approximately **101,500** in 2014. If the population growth rate remained consistent at **0.34% per year** (as observed between 2011-2014), the estimated population of Ancona in 2025 would be **105,351**. 

This is an unofficial projection based on historical data. For the most accurate and up-to-date figures, official census data or reports from the Italian National Institute of Statistics (ISTAT) would be required.


In [7]:
bot = ReACTAgent(system=system_prompt, tools=TOOLS, model="deepseek-reasoner")
response = bot(question, verbose=False)
print("Response:", response)

** CALLING FUNCTION search_web **
*** ARGS ***
query='Ancona population 2025 projection' max_results=10

** CALLING FUNCTION search_web **
*** ARGS ***
query='Ancona Italy population 2025 official estimate' max_results=10

** CALLING FUNCTION read_website **
*** ARGS ***
url='https://www.citypopulation.de/en/italy/cities/marche/'

Response: Based on the official population estimates from the Istituto Nazionale di Statistica Italia (Italy's National Institute of Statistics), the population of Ancona as of January 1, 2025 is **99,469**.

This information comes directly from the detailed city and commune population tables on citypopulation.de, which sources its data from Italy's official statistics agency. The table shows:
- 2021 census: 98,402
- 2025 estimate: 99,469

The population estimate represents the projected population for the city proper of Ancona as of the beginning of 2025.
